# Limpieza de Datos diabetic_data.csv
Este Notebook muestra los procedimientos utilizados para la limpieza de datos previa al análisis exploratorio.

In [ ]:
# Importar librerías relevantes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from google.colab import drive, files

# Mostrar todas las columnas para mejor visualización
pd.set_option('display.max_columns', None)

# Lectura inicial de datos
#!git clone https://github.com/Caanar/despliegue-prediccion-readmision.git
files.upload()
df = pd.read_csv("diabetic_data.csv")
#df = pd.read_csv("data/diabetic_data.csv")
print("Los datos originales tienen {} columnas y {} observaciones".format(df.shape[1], df.shape[0]))
display(df.head(3))
display(df.info())

Saving diabetic_data.csv to diabetic_data.csv
Los datos originales tienen 50 columnas y 101766 observaciones


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

None

De acuerdo con la documentación, las variables 'admission_type_id', 'discharge_disposition_id' y 'admission_source_id' deberían ser categoricas en vez de numericas. Además, en cada una de estas variables podemos ver distintas categorías para 'NULL', 'Not Mapped' y 'Unknown/Invalid', por lo que vamos a unir estas categorías en una sola.

In [ ]:
df['admission_type_id'] = df['admission_type_id'].replace({6:5, 8:5}).astype('str')
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace({25:18, 26:18}).astype('str')
df['admission_source_id'] = df['admission_source_id'].replace({15:17, 20:17}).astype('str')

## Datos Perdidos

In [ ]:
na = pd.DataFrame(df.isna().sum()).rename(columns={0:'# Null Values'})
display(na[na['# Null Values']>0])

,# Null Values


Acá podemos ver que pandas no logró identificar los valores perdidos. En la documentación se reporta que algunas columnas tienen valores perdidos. Una exploración más detallada nos revela que en este caso los datos perdidos toman el valor de '?'.

In [ ]:
na = {}
for e in df.columns:
    if df[e].dtype == object:
         if df[e][df[e] == '?'].count() > 0:
            na[e] = [df[e][df[e] == '?'].count(), df[e][df[e] == '?'].count()/df.shape[0]]
pd.DataFrame.from_dict(na).T.rename(columns={0:'# de datos perdidos', 1:'Como porcentaje'})

,# de datos perdidos,Como porcentaje
race,2273.0,0.022336
weight,98569.0,0.968585
payer_code,40256.0,0.395574
medical_specialty,49949.0,0.490822
diag_1,21.0,0.000206
diag_2,358.0,0.003518
diag_3,1423.0,0.013983


Ahora vemos que al menos 6 columnas tienen valores nulos. Vemos que payer_code, medical_specialty y weight tienen una cantidad muy alta de valores nulos, por lo que no los usaremos como parte de nuestros modelos, ya que cualquier intento de imputación podría afectar seriamente la varianza sin contribuir al rendimiento del modelo.

In [ ]:
df.drop(['weight','payer_code','medical_specialty','diag_1','diag_2','diag_3'], axis = 1, inplace=True)
df.replace({'?':np.nan}, inplace=True)

Ahora exploraremos otros patrones indeseables en los datos verificando las categorías posibles en cada variable categórica.

In [ ]:
for e in df.columns:
    if df[e].dtype == object:
        print(e, df[e].unique())

race ['Caucasian' 'AfricanAmerican' nan 'Other' 'Asian' 'Hispanic']
gender ['Female' 'Male' 'Unknown/Invalid']
age ['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
admission_type_id ['5' '1' '2' '3' '4' '7']
discharge_disposition_id ['18' '1' '3' '6' '2' '5' '11' '7' '10' '4' '14' '8' '13' '12' '16' '17'
 '22' '23' '9' '20' '15' '24' '28' '19' '27']
admission_source_id ['1' '7' '2' '4' '5' '6' '17' '3' '8' '9' '14' '10' '22' '11' '25' '13']
max_glu_serum ['None' '>300' 'Norm' '>200']
A1Cresult ['None' '>7' '>8' 'Norm']
metformin ['No' 'Steady' 'Up' 'Down']
repaglinide ['No' 'Up' 'Steady' 'Down']
nateglinide ['No' 'Steady' 'Down' 'Up']
chlorpropamide ['No' 'Steady' 'Down' 'Up']
glimepiride ['No' 'Steady' 'Down' 'Up']
acetohexamide ['No' 'Steady']
glipizide ['No' 'Steady' 'Up' 'Down']
glyburide ['No' 'Steady' 'Up' 'Down']
tolbutamide ['No' 'Steady']
pioglitazone ['No' 'Steady' 'Up' 'Down']
rosiglitazone ['No' 'Steady' 'Up' 'Down']
aca

Podemos ver que las características 'citoglipton' y 'examide' toman el mismo valor para todas las observaciones, por lo que no aportan información útil para predecir 'readmitido' y, por lo tanto, las eliminaremos. Además, según la documentación, 'encounter_id' y 'patient_nbr' son ID's, por lo que podemos eliminarlos de forma segura.

In [ ]:
df.drop(['citoglipton', 'examide', 'encounter_id', 'patient_nbr'], axis = 1, inplace=True)

Con esto podemos estar seguros de que no hay más datos perdidos y las variables se encuentran en un formato adecuado según sus dtypes.

In [ ]:
df.to_csv('diabetic_data_processed.csv')
files.download('diabetic_data_processed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>